![](./img/NLP_banner.jpg)

# 看图说话 （Image Captioning）
#### \[稀牛学院 x 网易云课程\]《AI工程师(自然语言处理方向)》 by胡俊杰

## 本章概述
    1.1 “看图说话”问题介绍
    1.2 简易CNN+RNN编码解码模型完成图片短文本描述原理
    1.3 注意力模型与“看图说话”优化
    1.4 【实战】基于CNN+RNN的编解码“看图说话”与beam-search优化
    1.5 【实战】基于attention model的“看图说话”实现


## 1.1 “看图说话”问题介绍
- image captioning 任务的定义是对一幅图片生成相对于图片内容的文本描述。一个AI系统不仅需要对图片进行识别，也需要理解和解释看到的图片内容，并且能够像人一样描述出图片中的对象之间的关系。
- 输入：一张图片
- 输出：一句文本描述


## 1.1 “看图说话”问题介绍
- 最早的image captioning 系统是2014年Circa 提出的，该系统使用多层感知系统（multi-layer perceptron MLP)将图片特征和词向量特征，去输出一串连贯的文字描述。
- 可以将image caption与机器翻译做一个类比：
    - image caption的输入是一张图片
    - machine translation的输入是一句外语句子
    - 两者的输出都是一个句子

## 1.1 “看图说话”问题介绍
- 常见的image captioning 系统的评估指标使用的是
    - BLEU， 是常见的机器翻译系统的评估指标，计算的是一句预测的文字与人类标注的参考文字之间的n-gram 重合度（overlap）。
    - METEOR， 也是常见的机器翻译系统的评估指标，其通过建立一个短语词表（phrase table），考虑了输出文本是否使用了相似短语。
    - CIDEr， 考虑了句子中的文字与图片的相关性
    - ROUGE-L，是text summerization的评估指标

## 1.1 “看图说话”问题介绍
- 常见的image captioning 系统的标准测试数据集包括
    - Flickr 8k
    - Flickr 30k
    - MS COCO

## 1.2 简易CNN+RNN编码解码模型完成图片短文本描述原理
- 常见的image captioning 系统是由一个CNN+RNN的编码解码模型完成
- 类比一下machine translation系统，通常由一个RNN encoder + RNN decoder组成
![](./img/rnn-rnn.png)
- image captioning系统，通常由一个CNN encoder + RNN decoder组成
![](./img/cnn-rnn.png)


## 1.2 简易CNN+RNN编码解码模型完成图片短文本描述原理
- CNN 能够提取一张图片的特征，其特征能用来做图片分类，目标识别，图片分割，及其他视觉任务。
- Vinyals et al. (2014) Show and Tell: A Neural Image Caption Generator 这篇文章将seq2seq模型中的LSTM encoder换成CNN encoder，用于提取图片的信息，得到一个固定长度的内容向量（context vector），之后通过一个RNN decoder，将信息使用文字的方式解码出来。
![](./img/cnn-lstm.png)

## 1.2 简易CNN+RNN编码解码模型完成图片短文本描述原理
- Donahue et al. (2014) Long-term Recurrent Convolutional Networks for Visual Recognition and Description 这篇文章使用了VGG Net作为CNN 去提取图片信息，在输入到一个LSTM decoder中输出文本。同时该文章还将这项技术应用到video captioning中。
![](./img/cnn-lstm-video.png)

## 1.2 简易CNN+RNN编码解码模型完成图片短文本描述原理
- 对比视频识别，看图说话，看视频说话
![](./img/img-video.png)

## 1.2 另一种完成图片短文本描述原理
- Fang et al 2014， From Captions to Visual Concepts and Back, 提供了另一个image caption系统的思路。
    1. 预测文字： 使用一个CNN去做目标识别，并且根据bounding box生成可能出现的文字
    2. 生成句子：通过一个统计语言模型，生成很多个可能的句子集合
    3. 重新排序已经生成的句子： 通过学习一个Deep Multimodal Similarity Model （DMSM）去重新排序所有可能的句子集合，取最高分数的句子作为系统输出。
   
![](./img/ms.png)

## 1.3 注意力机制与beam search优化
- 注意力机制 - Kelvin et al. (2014) Show, Attend and Tell: Neural Image Caption Generation with Visual Attention
    - 类比人看图说话：当人在解说一幅图片的时候，每预测一个字，会关注到图片上的不同位置。
    - 在解码器预测文字的时候，会关注到跟当前文字内容和图片最相关的位置。
    - 举例：a woman standing in a living room holding a Wii remote .
![](./img/example.gif)

## 1.3 注意力机制与beam search优化
- soft attention: 对一张图片中每个关注的地方都有一个关注权重，这个权重是介于0和1之间的数值。 
- hard attention: 对一张图片中每个关注的地方都有一个关注权重，这个权重是0或者1，1表示关注该区域。
![](./img/example1.png)

## 1.3 注意力机制与beam search优化
- 关注的名词会对应于图片中高亮的位置
![](./img/example2.png)

## 1.3 注意力机制与beam search优化
- 注意力机制 
    - 一张图片作为输入，CNN不仅可以通过最后一层全连接层（fully-connected layer）抓取出一个固定长度的向量，也即是context vector。还可以抓取出中间卷积层中的向量
![](./img/cnn.jpeg)

## 1.3 注意力机制与beam search优化
- 注意力机制 
    - 一张图片的卷积层中的向量有14x14=196个feature maps $a_i, i=1...196$，每个feature map对应于每个图片中不同的高亮位置。
    - 注意力机制通过计算每个feature map与当前的hidden state计算两者之间的相关度，这里的hidden state $h_{t-1}$ 总结了已经生成的1到t-1个单词的内容。
    $$e_{ti}=f_{att}(a_i, h_{t-1}) \\ \alpha_{ti} = {\exp(e_{ti}) \over \sum_{k=1}^L \exp(e_{tk}) }$$
    - 之后通过加权求和得到注意力内容向量 $\hat{z}_t$。
    $$\hat{z}_t=\phi(\{a_i\}, \{\alpha_i\})$$
![](./img/show_attend.png)

## 1.3 注意力机制与beam search优化
- 注意力机制 
    - 通过将196个feature maps求平均值去初始化LSTM中的 memory cell $c_0, h_0$ 
    - 根据图片及已经生成的部分单词，去预测下一个单词
    $$c_0 = f_{init,c}({1\over L} \sum_i^L a_i) \\ h_0 = f_{init,h}({1\over L} \sum_i^L a_i) \\  p(y_t|a, y_1^{t-1}) \propto \exp(L_o (E y_{t-1} + L_h h_t + L_z \hat{z}_t))$$
![](./img/show_attend.png)

## 1.3 注意力机制与beam search优化
- beam search 优化
    - 每次预测下一个单词的时候，计算当前所有路径的data log-likelihood并进行排序, 只保留data log-likelihood 最大值的K个beams。
![](./img/beam.png)

## 1.4 【实战】基于CNN+RNN的编解码“看图说话”与beam-search优化
- tensorflow 官方源代码：https://github.com/tensorflow/models/tree/master/research/im2txt
    - Bazel (http://bazel.io/docs/install.html)
    - Python 2.7
    - TensorFlow 1.0 or greater (https://www.tensorflow.org/install/)
    - NumPy (http://www.scipy.org/install.html)
    - Natural Language Toolkit (NLTK):
        - NLTK (http://www.nltk.org/install.html)
        - install the NLTK data package "punkt" (http://www.nltk.org/data.html)
    - Unzip
- 可以使用conda安装环境
```bash 
conda env create --file im2txt/conda-env/ubuntu-18-04-environment.yaml
conda activate im2txt
```

In [ ]:
%%bash
! git clone https://github.com/tensorflow/models.git

! home="./models/research/"    #文件夹models存放地址
! MSCOCO_DIR="$home/im2txt/data/mscoco"

# 预测脚本
! cd $home/im2txt
! bazel build //im2txt:download_and_preprocess_mscoco

# 下载MSCOCO数据并进行数据预处理
! bazel-bin/im2txt/download_and_preprocess_mscoco "${MSCOCO_DIR}"


In [1]:
%%bash
! home="$PWD/models/research/"
! MSCOCO_DIR="$home/im2txt/data/mscoco"

# 保存 Inception v3 checkpoint的地址
! INCEPTION_DIR="$home/im2txt/data"
! mkdir -p ${INCEPTION_DIR}

# 下载预训练好的CNN模型
! wget "http://download.tensorflow.org/models/inception_v3_2016_08_28.tar.gz"
! tar -xvf "inception_v3_2016_08_28.tar.gz" -C ${INCEPTION_DIR}
! rm "inception_v3_2016_08_28.tar.gz"

inception_v3.ckpt


fatal: destination path 'models' already exists and is not an empty directory.
--2019-03-15 06:34:59--  http://download.tensorflow.org/models/inception_v3_2016_08_28.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 216.58.200.240, 2404:6800:4008:802::2010
Connecting to download.tensorflow.org (download.tensorflow.org)|216.58.200.240|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 100885009 (96M) [application/x-tar]
Saving to: 'inception_v3_2016_08_28.tar.gz'

     0K .......... .......... .......... .......... ..........  0%  440K 3m44s
    50K .......... .......... .......... .......... ..........  0% 1.01M 2m39s
   100K .......... .......... .......... .......... ..........  0% 1.60M 2m6s
   150K .......... .......... .......... .......... ..........  0% 1.91M 1m47s
   200K .......... .......... .......... .......... ..........  0% 3.07M 92s
   250K .......... .......... .......... .......... ..........  0% 3.53M 81s
   300K .......... ..

In [3]:
%%bash
! home="./models/research/"

# Directory containing preprocessed MSCOCO data.
! MSCOCO_DIR="${home}/im2txt/data/mscoco"

# Inception v3 checkpoint file.
! INCEPTION_CHECKPOINT="${home}/im2txt/data/inception_v3.ckpt"

# 保存模型位置
! MODEL_DIR="${home}/im2txt/model"

# 创建模型
! cd $home/im2txt
! bazel build -c opt //im2txt/...

# 训练模型
! bazel-bin/im2txt/train \
  --input_file_pattern="${MSCOCO_DIR}/train-?????-of-00256" \
  --inception_checkpoint_file="${INCEPTION_CHECKPOINT}" \
  --train_dir="${MODEL_DIR}/train" \
  --train_inception=false \
  --number_of_steps=1000000

Starting local Bazel server and connecting to it...
Loading: 
Loading: 0 packages loaded
Analyzing: 17 targets (3 packages loaded)
Analyzing: 17 targets (3 packages loaded, 0 targets configured)
Analyzing: 17 targets (10 packages loaded, 45 targets configured)
Analyzing: 17 targets (18 packages loaded, 90 targets configured)
Analyzing: 17 targets (20 packages loaded, 189 targets configured)
Analyzing: 17 targets (20 packages loaded, 189 targets configured)
Analyzing: 17 targets (20 packages loaded, 189 targets configured)
Analyzing: 17 targets (20 packages loaded, 189 targets configured)
Analyzing: 17 targets (20 packages loaded, 189 targets configured)
Analyzing: 17 targets (20 packages loaded, 189 targets configured)
Analyzing: 17 targets (20 packages loaded, 189 targets configured)
Analyzing: 17 targets (20 packages loaded, 189 targets configured)
Analyzing: 17 targets (20 packages loaded, 189 targets configured)
Analyzing: 17 targets (20 packages loaded, 189 targets configured)
INF

In [ ]:
%%bash
! home="./models/research/"
# 使用模型来进行预测 
! CHECKPOINT_PATH="${home}/im2txt/model/train"

# 词表位置
! VOCAB_FILE="${home}/im2txt/data/mscoco/word_counts.txt"

# 使用某一张JPEG图片来做演示
! IMAGE_FILE="${home}/im2txt/data/mscoco/raw-data/val2014/COCO_val2014_000000224477.jpg"

# 创建inference的二进制代码
! cd $home/im2txt
! bazel build -c opt //im2txt:run_inference

! export CUDA_VISIBLE_DEVICES=0

# 生成caption
! bazel-bin/im2txt/run_inference \
  --checkpoint_path=${CHECKPOINT_PATH} \
  --vocab_file=${VOCAB_FILE} \
  --input_files=${IMAGE_FILE}



```bash
Captions for image COCO_val2014_000000224477.jpg:
  0) a man riding a wave on top of a surfboard . (p=0.040413)
  1) a person riding a surf board on a wave (p=0.017452)
  2) a man riding a wave on a surfboard in the ocean . (p=0.005743)
```

![](./img/example_img.jpg)

## 1.3 注意力机制与beam search优化

```python
class ShowAndTellModel(object):
  def __init__(self, config, mode, train_inception=False):
    """Basic setup.
    Args:
      config: Object containing configuration parameters.
      mode: "train", "eval" or "inference".
      train_inception: Whether the inception submodel variables are trainable.
    """
    ...
    # A float32 Tensor with shape [batch_size, height, width, channels].
    self.images = None

    # An int32 Tensor with shape [batch_size, padded_length].
    self.input_seqs = None

    # An int32 Tensor with shape [batch_size, padded_length].
    self.target_seqs = None

    # An int32 0/1 Tensor with shape [batch_size, padded_length].
    self.input_mask = None

    # A float32 Tensor with shape [batch_size, embedding_size].
    self.image_embeddings = None

    # A float32 Tensor with shape [batch_size, padded_length, embedding_size].
    self.seq_embeddings = None

    # A float32 scalar Tensor; the total loss for the trainer to optimize.
    self.total_loss = None

    # A float32 Tensor with shape [batch_size * padded_length].
    self.target_cross_entropy_losses = None

    # A float32 Tensor with shape [batch_size * padded_length].
    self.target_cross_entropy_loss_weights = None

    # Collection of variables from the inception submodel.
    self.inception_variables = []

    # Function to restore the inception submodel from checkpoint.
    self.init_fn = None
    
  def build_inputs(self):
    """预处理输入数据
    Outputs:
      self.images
      self.input_seqs
      self.target_seqs (training and eval only)
      self.input_mask (training and eval only)
    """
    if self.mode == "inference":
      # In inference mode, images and inputs are fed via placeholders.
      image_feed = tf.placeholder(dtype=tf.string, shape=[], name="image_feed")
      input_feed = tf.placeholder(dtype=tf.int64,
                                  shape=[None],  # batch_size
                                  name="input_feed")

      # Process image and insert batch dimensions.
      images = tf.expand_dims(self.process_image(image_feed), 0)
      input_seqs = tf.expand_dims(input_feed, 1)

      # No target sequences or input mask in inference mode.
      target_seqs = None
      input_mask = None
    else:
      # Prefetch serialized SequenceExample protos.
      input_queue = input_ops.prefetch_input_data(
          self.reader,
          self.config.input_file_pattern,
          is_training=self.is_training(),
          batch_size=self.config.batch_size,
          values_per_shard=self.config.values_per_input_shard,
          input_queue_capacity_factor=self.config.input_queue_capacity_factor,
          num_reader_threads=self.config.num_input_reader_threads)

      # Image processing and random distortion. Split across multiple threads
      # with each thread applying a slightly different distortion.
      assert self.config.num_preprocess_threads % 2 == 0
      images_and_captions = []
      for thread_id in range(self.config.num_preprocess_threads):
        serialized_sequence_example = input_queue.dequeue()
        encoded_image, caption = input_ops.parse_sequence_example(
            serialized_sequence_example,
            image_feature=self.config.image_feature_name,
            caption_feature=self.config.caption_feature_name)
        image = self.process_image(encoded_image, thread_id=thread_id)
        images_and_captions.append([image, caption])

      # Batch inputs.
      queue_capacity = (2 * self.config.num_preprocess_threads *
                        self.config.batch_size)
      images, input_seqs, target_seqs, input_mask = (
          input_ops.batch_with_dynamic_pad(images_and_captions,
                                           batch_size=self.config.batch_size,
                                           queue_capacity=queue_capacity))

    self.images = images
    self.input_seqs = input_seqs
    self.target_seqs = target_seqs
    self.input_mask = input_mask

  def build_image_embeddings(self):
    """CNN 将图片encode到图像空间
    Inputs:
      self.images
    Outputs:
      self.image_embeddings
    """
    inception_output = image_embedding.inception_v3(
        self.images,
        trainable=self.train_inception,
        is_training=self.is_training())
    self.inception_variables = tf.get_collection(
        tf.GraphKeys.GLOBAL_VARIABLES, scope="InceptionV3")

    # Map inception output into embedding space.
    with tf.variable_scope("image_embedding") as scope:
      image_embeddings = tf.contrib.layers.fully_connected(
          inputs=inception_output,
          num_outputs=self.config.embedding_size,
          activation_fn=None,
          weights_initializer=self.initializer,
          biases_initializer=None,
          scope=scope)

    # Save the embedding size in the graph.
    tf.constant(self.config.embedding_size, name="embedding_size")

    self.image_embeddings = image_embeddings

  def build_seq_embeddings(self):
    """将输入文字映射到词向量空间
    Inputs:
      self.input_seqs
    Outputs:
      self.seq_embeddings
    """
    with tf.variable_scope("seq_embedding"), tf.device("/cpu:0"):
      embedding_map = tf.get_variable(
          name="map",
          shape=[self.config.vocab_size, self.config.embedding_size],
          initializer=self.initializer)
      seq_embeddings = tf.nn.embedding_lookup(embedding_map, self.input_seqs)

    self.seq_embeddings = seq_embeddings

  def build_model(self):
    """创建 CNN-encoder 与RNN-decoder，计算loss 
    Inputs:
      self.image_embeddings
      self.seq_embeddings
      self.target_seqs (training and eval only)
      self.input_mask (training and eval only)
    Outputs:
      self.total_loss (training and eval only)
      self.target_cross_entropy_losses (training and eval only)
      self.target_cross_entropy_loss_weights (training and eval only)
    """
    # This LSTM cell has biases and outputs tanh(new_c) * sigmoid(o), but the
    # modified LSTM in the "Show and Tell" paper has no biases and outputs
    # new_c * sigmoid(o).
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(
        num_units=self.config.num_lstm_units, state_is_tuple=True)
    if self.mode == "train":
      lstm_cell = tf.contrib.rnn.DropoutWrapper(
          lstm_cell,
          input_keep_prob=self.config.lstm_dropout_keep_prob,
          output_keep_prob=self.config.lstm_dropout_keep_prob)

    with tf.variable_scope("lstm", initializer=self.initializer) as lstm_scope:
      # Feed the image embeddings to set the initial LSTM state.
      zero_state = lstm_cell.zero_state(
          batch_size=self.image_embeddings.get_shape()[0], dtype=tf.float32)
      _, initial_state = lstm_cell(self.image_embeddings, zero_state)

      # Allow the LSTM variables to be reused.
      lstm_scope.reuse_variables()

      if self.mode == "inference":
        # In inference mode, use concatenated states for convenient feeding and
        # fetching.
        tf.concat(axis=1, values=initial_state, name="initial_state")

        # Placeholder for feeding a batch of concatenated states.
        state_feed = tf.placeholder(dtype=tf.float32,
                                    shape=[None, sum(lstm_cell.state_size)],
                                    name="state_feed")
        state_tuple = tf.split(value=state_feed, num_or_size_splits=2, axis=1)

        # Run a single LSTM step.
        lstm_outputs, state_tuple = lstm_cell(
            inputs=tf.squeeze(self.seq_embeddings, axis=[1]),
            state=state_tuple)

        # Concatentate the resulting state.
        tf.concat(axis=1, values=state_tuple, name="state")
      else:
        # Run the batch of sequence embeddings through the LSTM.
        sequence_length = tf.reduce_sum(self.input_mask, 1)
        lstm_outputs, _ = tf.nn.dynamic_rnn(cell=lstm_cell,
                                            inputs=self.seq_embeddings,
                                            sequence_length=sequence_length,
                                            initial_state=initial_state,
                                            dtype=tf.float32,
                                            scope=lstm_scope)

    # Stack batches vertically.
    lstm_outputs = tf.reshape(lstm_outputs, [-1, lstm_cell.output_size])

    with tf.variable_scope("logits") as logits_scope:
      logits = tf.contrib.layers.fully_connected(
          inputs=lstm_outputs,
          num_outputs=self.config.vocab_size,
          activation_fn=None,
          weights_initializer=self.initializer,
          scope=logits_scope)

    if self.mode == "inference":
      tf.nn.softmax(logits, name="softmax")
    else:
      targets = tf.reshape(self.target_seqs, [-1])
      weights = tf.to_float(tf.reshape(self.input_mask, [-1]))

      # Compute losses.
      losses = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=targets,
                                                              logits=logits)
      batch_loss = tf.div(tf.reduce_sum(tf.multiply(losses, weights)),
                          tf.reduce_sum(weights),
                          name="batch_loss")
      tf.losses.add_loss(batch_loss)
      total_loss = tf.losses.get_total_loss()

      # Add summaries.
      tf.summary.scalar("losses/batch_loss", batch_loss)
      tf.summary.scalar("losses/total_loss", total_loss)
      for var in tf.trainable_variables():
        tf.summary.histogram("parameters/" + var.op.name, var)

      self.total_loss = total_loss
      self.target_cross_entropy_losses = losses  # Used in evaluation.
      self.target_cross_entropy_loss_weights = weights  # Used in evaluation.
```

## 1.5 【实战】基于attention model的“看图说话”实现

```bash
# 下载代码
git clone https://github.com/yunjey/show-attend-and-tell-tensorflow.git
git clone https://github.com/tylin/coco-caption.git

# 下载数据
# 所有图片存放在 image/ 这个文件夹下，VGGNet19模型文件会存放在 data/ 文件夹下
cd show-attend-and-tell-tensorflow
pip install -r requirements.txt
chmod +x ./download.sh
./download.sh

# 预处理数据
# 把所有图片都resize成224x224维度, 存放在image/train2014_resized/， image/val2014_resized/
python resize.py
# 把图片和文字都预处理成向量
python prepro.py

# 训练模型
python train.py
```

## 1.5 【实战】基于attention model的“看图说话”实现

```python
class CaptionGenerator(object):
    def __init__(self, word_to_idx, dim_feature=[196, 512], dim_embed=512, dim_hidden=1024, n_time_step=16,
                  prev2out=True, ctx2out=True, alpha_c=0.0, selector=True, dropout=True):
        """
        Args:
            word_to_idx: word-to-index mapping dictionary.
            dim_feature: (optional) Dimension of vggnet19 conv5_3 feature vectors.
            dim_embed: (optional) Dimension of word embedding.
            dim_hidden: (optional) Dimension of all hidden state.
            n_time_step: (optional) Time step size of LSTM.
            prev2out: (optional) previously generated word to hidden state. (see Eq (7) for explanation)
            ctx2out: (optional) context to hidden state (see Eq (7) for explanation)
            alpha_c: (optional) Doubly stochastic regularization coefficient. (see Section (4.2.1) for explanation)
            selector: (optional) gating scalar for context vector. (see Section (4.2.1) for explanation)
            dropout: (optional) If true then dropout layer is added.
        """

        self.word_to_idx = word_to_idx
        self.idx_to_word = {i: w for w, i in word_to_idx.iteritems()}
        self.prev2out = prev2out
        self.ctx2out = ctx2out
        self.alpha_c = alpha_c
        self.selector = selector
        self.dropout = dropout
        self.V = len(word_to_idx)
        self.L = dim_feature[0]
        self.D = dim_feature[1]
        self.M = dim_embed
        self.H = dim_hidden
        self.T = n_time_step
        self._start = word_to_idx['<START>']
        self._null = word_to_idx['<NULL>']

        self.weight_initializer = tf.contrib.layers.xavier_initializer()
        self.const_initializer = tf.constant_initializer(0.0)
        self.emb_initializer = tf.random_uniform_initializer(minval=-1.0, maxval=1.0)

        # Place holder for features and captions
        self.features = tf.placeholder(tf.float32, [None, self.L, self.D])
        self.captions = tf.placeholder(tf.int32, [None, self.T + 1])


    def build_model(self):
        features = self.features
        captions = self.captions
        batch_size = tf.shape(features)[0]

        captions_in = captions[:, :self.T]
        captions_out = captions[:, 1:]
        mask = tf.to_float(tf.not_equal(captions_out, self._null))


        # batch normalize feature vectors
        features = self._batch_norm(features, mode='train', name='conv_features')

        c, h = self._get_initial_lstm(features=features)
        x = self._word_embedding(inputs=captions_in)
        features_proj = self._project_features(features=features)

        loss = 0.0
        alpha_list = []
        lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units=self.H)

        for t in range(self.T):
            context, alpha = self._attention_layer(features, features_proj, h, reuse=(t!=0))
            alpha_list.append(alpha)

            if self.selector:
                context, beta = self._selector(context, h, reuse=(t!=0))

            with tf.variable_scope('lstm', reuse=(t!=0)):
                _, (c, h) = lstm_cell(inputs=tf.concat( [x[:,t,:], context],1), state=[c, h])

            logits = self._decode_lstm(x[:,t,:], h, context, dropout=self.dropout, reuse=(t!=0))

            loss += tf.reduce_sum(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=captions_out[:, t],logits=logits)*mask[:, t] )

        if self.alpha_c > 0:
            alphas = tf.transpose(tf.stack(alpha_list), (1, 0, 2))     # (N, T, L)
            alphas_all = tf.reduce_sum(alphas, 1)      # (N, L)
            alpha_reg = self.alpha_c * tf.reduce_sum((16./196 - alphas_all) ** 2)
            loss += alpha_reg

        return loss / tf.to_float(batch_size)
   
```

## 1.5 【实战】基于attention model的“看图说话”实现

- 测试生成caption

In [2]:
import matplotlib.pyplot as plt
#import cPickle as pickle
import tensorflow as tf
from core.solver import CaptioningSolver
from core.model import CaptionGenerator
from core.utils import load_coco_data
from core.bleu import evaluate

%matplotlib inline
plt.rcParams['figure.figsize'] = (8.0, 6.0)  # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

ImportError: No module named 'core'

In [ ]:
import matplotlib.pyplot as plt
import cPickle as pickle
import tensorflow as tf
from core.solver import CaptioningSolver
from core.model import CaptionGenerator
from core.utils import load_coco_data
from core.bleu import evaluate

%matplotlib inline
plt.rcParams['figure.figsize'] = (8.0, 6.0)  # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

In [ ]:
data = load_coco_data(data_path='./data', split='val')
test = load_coco_data(data_path='./data', split='test')
with open('./data/train/word_to_idx.pkl', 'rb') as f:
    word_to_idx = pickle.load(f)

In [ ]:
model = CaptionGenerator(word_to_idx, dim_feature=[196, 512], dim_embed=512,
                        dim_hidden=1500, n_time_step=16, prev2out=True, 
                        ctx2out=True, alpha_c=1.0, selector=True, dropout=True)

In [ ]:
solver = CaptioningSolver(model, data, data, n_epochs=15, batch_size=128, update_rule='adam',
                        learning_rate=0.0025, print_every=2000, save_every=1, image_path='./image/val2014_resized',
                        pretrained_model=None, model_path='./model/lstm', test_model='./model/lstm3/model-18',
                        print_bleu=False, log_path='./log/')
tf.get_variable_scope().reuse_variables()
solver.test(test, split='test')

```bash
Sampled Caption: a sign that is on a pole in front of a building .
```
![](./img/test1.png)

## 本章小结
    1.1 “看图说话”问题介绍
    1.2 简易CNN+RNN编码解码模型完成图片短文本描述原理
    1.3 注意力模型与“看图说话”优化
    1.4 【实战】基于CNN+RNN的编解码“看图说话”与beam-search优化
    1.5 【实战】基于attention model的“看图说话”实现
![](./img/xiniu_neteasy.png)